<a href="https://colab.research.google.com/github/ekgren/workshop/blob/main/Day2/QA_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers

In [ ]:
import datasets
import transformers
import torch
import copy
import tqdm

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
kb_bert = transformers.AutoModel.from_pretrained('KB/bert-base-swedish-cased')

In [ ]:
dataset = datasets.load_dataset('stsb_mt_sv')
train_ds = dataset['train']
test_ds = dataset['test']
eval_ds = dataset['validation']


In [ ]:
def encode(*texts):
  assert 1 <= len(texts) <= 2
  return tokenizer(*texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

In [ ]:
train_ds[0]

In [ ]:
def collate_paired(rows):
  s1s = [row['sentence1'] for row in rows]
  s2s = [row['sentence2'] for row in rows]
  scores = torch.tensor([row['score'] for row in rows])
  return encode(s1s, s2s), scores

def collate_dual(rows):
  s1s = [row['sentence1'] for row in rows]
  s2s = [row['sentence2'] for row in rows]
  scores = torch.tensor([row['score'] for row in rows])
  return encode(s1s), encode(s2s), scores


In [ ]:
COLLATER = collate_paired
train_dl = torch.utils.data.DataLoader(  
    train_ds,
    collate_fn=COLLATER,
    shuffle=True,
    batch_size=8,
    pin_memory=True,
  )
test_dl = torch.utils.data.DataLoader(  
    test_ds,
    collate_fn=COLLATER,
    shuffle=False,
    batch_size=8,
    pin_memory=True,
  )
eval_dl = torch.utils.data.DataLoader(  
    eval_ds,
    collate_fn=COLLATER,
    shuffle=False,
    batch_size=8,
    pin_memory=True,
  )

In [ ]:

class PairedModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.model = copy.deepcopy(kb_bert)
    self.linear = torch.nn.Linear(768, 1)
  
  def forward(self, data):
    return self.linear(self.model(**data)['pooler_output']).squeeze(-1)

In [ ]:

batch, score = next(iter(train_dl))

In [ ]:

paired_model(batch)

In [ ]:
paired_model = PairedModel().cuda()
optimizer = torch.optim.AdamW(paired_model.parameters())
EPOCHS = 1
for epoch in range(EPOCHS):
  batches = tqdm.tqdm(train_dl)
  for batch, score in batches:
    batch = {k : v.cuda() for k, v in batch.items()}
    score = score.cuda()
    optimizer.zero_grad()

    prediction = paired_model(batch)
    
    loss = torch.nn.functional.mse_loss(prediction, score)

    loss.backward()

    optimizer.step()

    batches.set_description('{:.2f}'.format(loss.item()))


# Namnigenkänning (Named Entity Recognition)

Exempel: Namnigenkänning

Kort förklaring av namnigenkänning

BERT base fine-tuned for Swedish NER. This model is fine-tuned on the SUC 3.0 dataset.

Entity types used are TME for time, PRS for personal names, LOC for locations, EVN for events and ORG for organisations.

In [ ]:
from transformers import pipeline

nlp = pipeline('ner', model='KB/bert-base-swedish-cased-ner', tokenizer='KB/bert-base-swedish-cased-ner')

In [ ]:
nlp('Hej jag heter Arne och jag vill byta lösenord.')

# Exempel KB-Bert ordmaskning

Maskar ord med Bert

https://github.com/af-ai-center/SweBERT/blob/master/getting_started_with_swebert.ipynb